In [55]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [19]:
html = requests.get("https://sfedu.ru/www/stat_pages22.show?p=EDU/N14116/D").text

In [57]:
with open("table.html", "r") as file:
    html = file.read()

In [87]:
soup = BeautifulSoup(html)


def get_univercitites():
    for table_row in soup.find_all("tr"):
        cells = table_row.find_all("td")

        if re.match(r'\d{2}\.\d{2}\.\d{2}', cells[0].text) != None:
            ed_id = cells[0].text            
            doc_links = [el.attrs["href"] for el in table_row.find_all("a") if "Документы" in el.text]

            if len(doc_links) > 0:
                yield (ed_id, doc_links[0])

In [92]:
univercities = list(get_univercitites())

In [110]:
def gen_dfs():
    for ed_id, link in univercities:
        df = pd.read_html(link)[0]
        df["ed_id"] = ed_id
        yield df

dt = pd.concat(gen_dfs())
dt.to_csv("disciplines_sfedu_extended.csv", index=False)
# df = pd.concat([["ORG"] = ed_id for ed_id, link in univercities])
# df.to_csv("disciplines_sfedu.csv", index=False)
# dfs = pd.read_html(doc_links[2])
# dfs[0]